<a href="https://www.kaggle.com/code/averma111/pytorch-ps-s3e14-v1?scriptVersionId=128615082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [47]:
%%capture
!pip install flaml

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [60]:
ROOT_DIR='/kaggle/input/playground-series-s3e14'
ORIGINAL_DIR='/kaggle/input/wild-blueberry-yield-prediction/Data in Brief/Data in Brief/WildBlueberryPollinationSimulationData.csv'
train_file = 'train.csv'
origianl ='WildBlueberryPollinationSimulationData.csv'

In [61]:
def preprocessing():
    train = pd.read_csv(ROOT_PATH+'/'+train_file)
    original_df = pd.read_csv(ORIGINAL_DIR)
    train.drop('id',axis=1,inplace=True)
    original_df.drop('Row#',axis=1,inplace=True)
    df_full = pd.concat([train, original_df])
    for col in original_df.columns:
        df_full[col] = df_full[col].astype('float64')
    df_full = df_full.drop_duplicates()
    
    return df_full

In [62]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    summ['inf'] = np.isinf(df).sum().sum()
    summ['duplicate'] = df.duplicated().sum()
    return summ

In [64]:
df_full=preprocessing()

In [65]:
summary('full',df_full)

full shape: (16059, 17)


,dtypes,null,unique,min,median,max,mean,std,inf,duplicate
clonesize,float64,0,6,10.000000,25.000000,40.000000,19.657824,6.615589,0,0
honeybee,float64,0,7,0.000000,0.500000,18.430000,0.390628,0.413323,0,0
bumbles,float64,0,12,0.000000,0.250000,0.585000,0.286556,0.060249,0,0
andrena,float64,0,17,0.000000,0.500000,0.750000,0.491548,0.148837,0,0
osmia,float64,0,16,0.000000,0.630000,0.750000,0.590906,0.141169,0,0
MaxOfUpperTRange,float64,0,6,69.700000,86.000000,94.600000,82.174955,9.149359,0,0
MinOfUpperTRange,float64,0,5,39.000000,52.000000,57.200000,49.674525,5.549016,0,0
AverageOfUpperTRange,float64,0,5,58.200000,71.900000,79.000000,68.659381,7.643815,0,0
MaxOfLowerTRange,float64,0,6,50.200000,62.000000,68.200000,59.233315,6.612713,0,0
MinOfLowerTRange,float64,0,7,24.300000,30.000000,33.000000,28.661947,3.196182,0,0


In [66]:
def feature_engineering(df):
    df["fruit_seed"] = df["fruitset"] * df["seeds"]
    df['insects'] = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']
    return df

In [68]:
df_final=feature_engineering(df_full)
df_final.head()

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield,fruit_seed,insects
0,25.0,0.50,0.25,0.75,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.425011,0.417545,32.460887,4476.81146,13.796231,2.00
1,25.0,0.50,0.25,0.50,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.444908,0.422051,33.858317,5548.12201,15.063846,1.75
2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.552927,0.470853,38.341781,6869.77760,21.200199,1.76
3,12.5,0.25,0.25,0.63,0.50,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.565976,0.478137,39.467561,6880.77590,22.337712,1.63
4,25.0,0.50,0.25,0.63,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.579677,0.494165,40.484512,7479.93417,23.467926,2.01


In [69]:
class RegressionBlueBerryNNet(torch.nn.Module):
    def __init__(self,input_features):
        super(RegressionBlueBerryNNet,self).__init__()
        self.input_layer = torch.nn.Linear(input_features,1)
        self.relu = torch.nn.ReLU()
        
    def forward(self,inputs):
        x = self.relu(self.input_layer(inputs))
        return x

In [70]:
from torch.utils.data import Dataset, DataLoader
# Train Data
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [72]:
def load_data(df_final):
    y = df_final['yield']
    df_final.drop(columns=['yield'],axis=1,inplace=True)
    X = df_final.copy()
    scaler = StandardScaler()
    numerical_cols = list(X.select_dtypes(include=['int','float']))
    X_numerical = scaler.fit_transform(X[numerical_cols].values)
    y=y.numpy()
    X_train,X_test,y_train,y_test = train_test_split(X_numerical,y,test_size=0.2,random_state=42)
    train_data = TrainData(torch.tensor(data=X_train,dtype=torch.float32,requires_grad=True),
                       torch.tensor(data=y_train,dtype=torch.float32,requires_grad=True))
    val_data = TrainData(torch.tensor(data=X_test,dtype=torch.float32,requires_grad=True),
                       torch.tensor(data=y_test,dtype=torch.float32,requires_grad=True))
    return train_data,val_data
    

In [ ]:
from ray import tune

def train_nnet(config, checkpoint_dir=None, data_dir=None):
    if "l1" not in config:
        logger.warning(config)
    net = Net(2**config["l1"], 2**config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.L1Loss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset,valset= load_data(data_dir)

    trainloader = DataLoader(dataset=trainset, batch_size=int(2**config["batch_size"], shuffle=True)
    valloader  =  DataLoader(dataset=valset, batch_size=nt(2**config["batch_size"])

    for epoch in range(int(round(config["num_epochs"]))):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = RegressionBlueBerryNNet(inputs.shape[1])
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = RegressionBlueBerryNNet(inputs.shape[1])
                #_, predicted = torch.max(outputs.data, 1)
                #total += labels.size(0)
                #correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1
                             
        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save(
                (net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps))
    print("Finished Training")

In [ ]:
def _test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
import numpy as np
import flaml
import os

data_dir = os.path.abspath("data")
load_data(data_dir)  # Download data for all trials before starting the run

In [ ]:
max_num_epoch = 100
config = {
    "l1": tune.randint(2, 9),   # log transformed with base 2
    "l2": tune.randint(2, 9),   # log transformed with base 2
    "lr": tune.loguniform(1e-4, 1e-1),
    "num_epochs": tune.loguniform(1, max_num_epoch),
    "batch_size": tune.randint(1, 5)    # log transformed with base 2
}

In [ ]:
import time
start_time = time.time()
result = flaml.tune.run(
    tune.with_parameters(train_cifar, data_dir=data_dir),
    config=config,
    metric="loss",
    mode="min",
    low_cost_partial_config={"num_epochs": 1},
    max_resource=max_num_epoch,
    min_resource=1,
    scheduler="asha",  # Use asha scheduler to perform early stopping based on intermediate results reported
    resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
    local_dir='logs/',
    num_samples=num_samples,
    time_budget_s=time_budget_s,
    use_ray=True)

In [ ]:
print(f"#trials={len(result.trials)}")
print(f"time={time.time()-start_time}")
best_trial = result.get_best_trial("loss", "min", "all")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.metric_analysis["loss"]["min"]))
print("Best trial final validation accuracy: {}".format(
    best_trial.metric_analysis["accuracy"]["max"]))

best_trained_model = Net(2**best_trial.config["l1"],
                         2**best_trial.config["l2"])
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device)

checkpoint_value = getattr(best_trial.checkpoint, "dir_or_data", None) or best_trial.checkpoint.value
checkpoint_path = os.path.join(checkpoint_value, "checkpoint")

model_state, optimizer_state = torch.load(checkpoint_path)
best_trained_model.load_state_dict(model_state)

test_acc = _test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))